In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time, os, datetime
from selenium import webdriver

def del_html_tag(raw_text):
    return BeautifulSoup(raw_text, "lxml").text

In [4]:
file_path = rf"{os.path.abspath('datas/pre_data/mango(1_226).txt')}"

with open(file_path, 'rt', encoding='UTF8') as f:
  lines = f.readlines()

lines = [line.rstrip('\n') for line in lines]

food_list = []
food_url = []

In [5]:
browser = webdriver.Chrome(rf"{os.path.abspath('utils/chromedriver')}")
for idx, data in enumerate(lines):
  page_number = 1
  run=True
  while run:
    # 처음 접속해서 받아오는 정보, 상세정보에 들어가기 위한 url, pagination
    browser.get(f'https://www.mangoplate.com/search/{data}?keyword={data}&page={str(page_number)}')
    time.sleep(5)
    last_page = 10
    
    html = browser.page_source
    soup = BeautifulSoup(html,'html.parser')
    figs = soup.find_all("figure", "restaurant-item")
    for i in range(len(figs)):
      food_list.append(figs[i].find("a")["href"])
    if(not figs or last_page == page_number):
      run = False
    else:
      page_number +=1

    #중복 제거
    for i in food_list:
      if i not in food_url:
        food_url.append(i)

C:\Users\r2com\AppData\Local\Temp\ipykernel_1788\3993428499.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(rf"{os.path.abspath('utils/chromedriver')}")


In [56]:
food_len = len(food_url)

name_list = [0] * food_len
rank_list = [0] * food_len

review_list = [0] * food_len
thumbnail_list = [0] * food_len

addr_list = [[0]] * food_len # 주소
call_list = [0] * food_len # 전화번호
type_list = [0] * food_len # 음식종류
pran_list = [0] * food_len # 가격대
park_list = [0] * food_len # 주차
time_list = [0] * food_len # 영업시간
break_list = [0] * food_len # 쉬는시간
holi_list = [0] * food_len # 휴일
menu_list = [0] * food_len # 메뉴
price_list = [0] * food_len # 메뉴

In [42]:
url = "https://www.mangoplate.com/restaurants/nWeCoq9KF7x6"
# print(food_url[i])
browser.get(url)
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

rank = soup.select_one('span > strong > span')
print(del_html_tag(str(rank)))
print(del_html_tag(str(rank)) if rank is not None else rank_list[20000])

None
0


In [55]:
food_url[:300]

['/restaurants/ByuIW33rXc',
 '/restaurants/iMRRP69qtkeO',
 '/restaurants/9EVyzauilukl',
 '/restaurants/ewTZ9Le7U54q',
 '/restaurants/4hAcTcdILi',
 '/restaurants/UjiK7BUxljkP',
 '/restaurants/ol9BImtmnFqo',
 '/restaurants/m16fiU1vHZhd',
 '/restaurants/eEMujXaKYLRJ',
 '/restaurants/QxTRZZv3Dl',
 '/restaurants/mfwd78vEtS',
 '/restaurants/koJ-1vE9dB',
 '/restaurants/jiQPoOdNDqQc',
 '/restaurants/AIodGGqVRYId',
 '/restaurants/B8CzA6i9Bb8Z',
 '/restaurants/2_sFMbsZC9HC',
 '/restaurants/Xg0LDOaBhaPX',
 '/restaurants/vcheyGIxPJ9D',
 '/restaurants/ugq5zpw24LjX',
 '/restaurants/ZO1izBuFxQ',
 '/restaurants/vnc0i7l-fd',
 '/restaurants/TmKdK3zz6fQs',
 '/restaurants/pTPRkiV93E',
 '/restaurants/lekDtZkiY1Cy',
 '/restaurants/IxPgfbX7wag7',
 '/restaurants/dbZtB1usggE2',
 '/restaurants/gxUqUtrD-IAa',
 '/restaurants/pI112Z4JVmBn',
 '/restaurants/TznMMYOSqMQK',
 '/restaurants/KyJsLQGd64',
 '/restaurants/S2ukxlPLTf_J',
 '/restaurants/5h2OL8a5GlY_',
 '/restaurants/U2YxVAmkOVod',
 '/restaurants/6IDQwmv2D9Nf'

In [57]:
for i in range(len(food_url[:300])):
  url = "https://www.mangoplate.com" + food_url[i]
  print(food_url[i])
  browser.get(url)
  time.sleep(2)
  html = browser.page_source
  soup = BeautifulSoup(html, 'html.parser')
  
  # 맛집의 상세정보(이름, 평점, 주소, 전화번호, 휴일, 메뉴, 이미지, 리뷰)
  # 이름
  title = soup.select_one('span > h1').get_text()
  sub_title = del_html_tag(str(soup.find("p", "branch")))
  if(len(sub_title) > 0):
    title += " " + sub_title
  # try:name_list.append(soup.select_one('span > h1').get_text())
  # except:name_list.append("")
  name_list[i] = title

  # 평점
  rank = soup.select_one('span > strong > span')
  # try:rank_list.append(soup.select_one('span > strong > span').get_text())
  # except:rank_list.append("")
  rank_list[i] = del_html_tag(str(rank)) if rank is not None else rank_list[i]

  # 정보
  tbody = soup.find('tbody')

  trs = tbody.find_all("tr")

  for j in range(len(trs)):
    info1 = trs[j].find("th", text="주소")
    if(info1 is not None):
      info1_tr = info1.parent
      addr = del_html_tag(str(info1_tr.find_all("td")[0])).split("지번")
      addrs = []
      for addr_ in addr:
        addrs.append(addr_.strip())
      addr_list[i] = addrs

    info2 = trs[j].find("th", text="전화번호")
    if(info2 is not None):
      info2_tr = info2.parent
      b = del_html_tag(str(info2_tr.find_all("td")))
      call_list[i] = b
    
    info3 = trs[j].find("th", text="음식 종류")
    if(info3 is not None):
      info3_tr = info3.parent
      c = del_html_tag(str(info3_tr.find_all("td")))
      type_list[i] = c[2:-2].split("/")
    
    info4 = trs[j].find("th", text="가격대")
    if(info4 is not None):
      info4_tr = info4.parent
      d = del_html_tag(str(info4_tr.find_all("td")))
      pran_list[i] = d
    
    info5 = trs[j].find("th", text="주차")
    if(info5 is not None):
      info5_tr = info5.parent
      e = del_html_tag(str(info5_tr.find_all("td")))
      park_list[i] = e
    
    info6 = trs[j].find("th", text="영업시간")
    if(info6 is not None):
      info6_tr = info6.parent
      f = del_html_tag(str(info6_tr.find_all("td")))
      time_list[i] = f
    
    info7 = trs[j].find("th", text="쉬는시간")
    if(info7 is not None):
      info7_tr = info7.parent
      g = del_html_tag(str(info7_tr.find_all("td")))
      break_list[i] = g
    
    info8 = trs[j].find("th", text="휴일")
    if(info8 is not None):
      info8_tr = info8.parent
      h = del_html_tag(str(info8_tr.find_all("td")))
      holi_list[i] = h
    
    info9 = trs[j].find("th", text="메뉴")
    if(info9 is not None):
      info9_tr = info9.parent
      j = info9_tr.find_all("td")[0]
      lis = j.find_all("li")
      menu = []
      price = []
      for li in lis:
        menu.append(li.find("span", "Restaurant_Menu").text)
        price.append(li.find("span", "Restaurant_MenuPrice").text)
      menu_list[i] = menu
      price_list[i] = price
  #리뷰
  review = soup.select_one('li:nth-child(1) > a > div.RestaurantReviewItem__ReviewContent > div > p')
  review_list[i] = review.get_text().strip() if review is not None else review_list[i]
  # try:review_list.append(review.get_text().strip())
  # except:review_list.append("")

  # 이미지
  img = ''
  try:
    img = soup.select_one('div:nth-child(1) > figure > figure > img')['src']
    # a = soup.select_one('div:nth-child(1) > figure > figure > img')['src']
  except TypeError:
    img = ''
  thumbnail_list[i] = img

  food_df = pd.DataFrame({'id':food_url,'name':name_list,'rank':rank_list,'review':review_list,'thumbnail':thumbnail_list, 'addr':addr_list, 'call':call_list, 'type':type_list, 'price':pran_list, 'park':park_list, 'time':time_list, 'break':break_list, 'holiday':holi_list, 'menu':menu_list})
  

/restaurants/ByuIW33rXc
/restaurants/iMRRP69qtkeO
/restaurants/9EVyzauilukl
/restaurants/ewTZ9Le7U54q
/restaurants/4hAcTcdILi
/restaurants/UjiK7BUxljkP
/restaurants/ol9BImtmnFqo
/restaurants/m16fiU1vHZhd
/restaurants/eEMujXaKYLRJ
/restaurants/QxTRZZv3Dl
/restaurants/mfwd78vEtS
/restaurants/koJ-1vE9dB
/restaurants/jiQPoOdNDqQc
/restaurants/AIodGGqVRYId
/restaurants/B8CzA6i9Bb8Z
/restaurants/2_sFMbsZC9HC
/restaurants/Xg0LDOaBhaPX
/restaurants/vcheyGIxPJ9D
/restaurants/ugq5zpw24LjX
/restaurants/ZO1izBuFxQ
/restaurants/vnc0i7l-fd
/restaurants/TmKdK3zz6fQs
/restaurants/pTPRkiV93E
/restaurants/lekDtZkiY1Cy
/restaurants/IxPgfbX7wag7
/restaurants/dbZtB1usggE2
/restaurants/gxUqUtrD-IAa
/restaurants/pI112Z4JVmBn
/restaurants/TznMMYOSqMQK
/restaurants/KyJsLQGd64
/restaurants/S2ukxlPLTf_J
/restaurants/5h2OL8a5GlY_
/restaurants/U2YxVAmkOVod
/restaurants/6IDQwmv2D9Nf
/restaurants/Rr3Nn2KvMQd-
/restaurants/4vX65o4FhZqB
/restaurants/KF2fMgRyP4x4
/restaurants/qDj4t75-9IGS
/restaurants/O9aEfZZUdJ
/resta

In [ ]:
food_df = pd.DataFrame({'id':food_url,'name':name_list,'rank':rank_list,'review':review_list,'thumbnail':thumbnail_list, 'addr':addr_list, 'call':call_list, 'type':type_list, 'price':pran_list, 'park':park_list, 'time':time_list, 'break':break_list, 'holiday':holi_list, 'menu':menu_list})
# %%
print(food_df)
print("[MANGO_RESTAURANT] data to csv file")

dt = datetime.datetime.now()
fName = f'datas/restaurant_mango_{dt.year}_{dt.month}_{dt.day}.csv'
# fName = f'crawling/datas/restaurant_mango_{dt.year}_{dt.month}_{dt.day}.csv'
fName = rf'{os.path.abspath(fName)}'

food_df.to_csv(fName, sep=',', encoding='utf-8-sig', index=False)